In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import torch


device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
from pathlib import Path
import pickle

from thera_panacea.utils import get_class_weights

with open(Path("data/cleaned_df.pkl"), "rb") as f:
    cleaned_df_ = pickle.load(f)

cleaned_df = cleaned_df_[["path", "label"]]

class_weight = get_class_weights(cleaned_df, device)

In [ ]:
import torch
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader
from torch.optim import Adam
from torch.nn import CrossEntropyLoss

from thera_panacea.dataset.dataset import TherasDS
from thera_panacea.model.baseline_model import BaselineModel, preprocess
from thera_panacea.trainer.trainer import Trainer


n_epochs = 25
batch_size = 2048

# Data
train_df, test_df = train_test_split(cleaned_df, test_size=.2)

train_ds = TherasDS(train_df, device, preprocess)
test_ds = TherasDS(test_df, device, preprocess)

train_dl = DataLoader(train_ds, batch_size=batch_size)
test_dl = DataLoader(test_ds, batch_size=batch_size, shuffle=False)

# Model
model = BaselineModel().to(device)

# Trainer
loss_function = CrossEntropyLoss(
    weight=get_class_weights(train_df, device),
    label_smoothing=0.1
)
opt = Adam(model.parameters())
model_path = Path(
    "cleaned_data_baseline_model_state_dict.pth")
trainer = Trainer(model, opt, loss_function, model_path)

# Train
trainer.train(train_dl, test_dl, n_epochs)

In [ ]:
from tqdm import tqdm


model = BaselineModel()
model.load_state_dict(torch.load(model_path, map_location="cpu"))
model.to("cuda")
model.eval()

preds = []
for imgs, labels in tqdm(test_dl):
    pred = model(imgs).argmax(-1)
    preds.extend(pred.cpu().numpy())

In [ ]:
import pickle
from thera_panacea.utils import get_results, display_results


labels = test_df['label']

results = get_results(preds, labels)
display_results(results)

with open(Path("cleaned_data_results.pkl"), "wb") as f:
    pickle.dump(results, f)